In [1]:
from pathlib import Path

import fsspec
import s3fs
import xarray as xr
import zarr

In [2]:
s3cfg = dict([l.split(' = ') for l in Path('/home/users/mmuetz/.s3cfg').read_text().split('\n') if l])

In [3]:
# Internal access allowed in notebooks - no https and s3-ext.
jasmin_s3 = s3fs.S3FileSystem(
    anon=False, secret=s3cfg['secret_key'],
    key=s3cfg['access_key'],
    client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'}
)
jasmin_s3.ls('sim-data')

['sim-data/10km_GAL9_nest',
 'sim-data/5km-RAL3',
 'sim-data/Africa_km4p4_RAL3P3.n1280_GAL9_nest',
 'sim-data/DYAMOND3_example_data',
 'sim-data/SAmer_km4p4_RAL3P3.n1280_GAL9_nest',
 'sim-data/SEA_km4p4_RAL3P3.n1280_GAL9_nest',
 'sim-data/dev']

In [4]:
def get_initialized_chunk_ids(jasmin_s3, url, variable):
    lsout = jasmin_s3.ls(f'{url}/{variable}/')
    fnames = [p.split('/')[-1] for p in lsout]
    chunk_ids_str = [p for p in fnames if not p.startswith('.z')]
    chunk_ids = sorted([tuple([int(v) for v in c.split('.')]) for c in chunk_ids_str])
    return chunk_ids

In [10]:
rel_url = 'sim-data/dev/{sim}/v4/data.healpix.{freq}.z{zoom}.zarr'.format(sim='glm.n2560_RAL3p3', freq='PT3H', zoom=10)
init_chunk_ids = get_initialized_chunk_ids(jasmin_s3, rel_url, 'ta')

In [11]:
init_chunk_ids

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 0, 4),
 (0, 0, 5),
 (0, 0, 6),
 (0, 0, 7),
 (0, 0, 8),
 (0, 0, 9),
 (0, 0, 10),
 (0, 0, 11),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 1, 6),
 (0, 1, 7),
 (0, 1, 8),
 (0, 1, 9),
 (0, 1, 10),
 (0, 1, 11),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 2, 6),
 (0, 2, 7),
 (0, 2, 8),
 (0, 2, 9),
 (0, 2, 10),
 (0, 2, 11),
 (0, 3, 0),
 (0, 3, 1),
 (0, 3, 2),
 (0, 3, 3),
 (0, 3, 4),
 (0, 3, 5),
 (0, 3, 6),
 (0, 3, 7),
 (0, 3, 8),
 (0, 3, 9),
 (0, 3, 10),
 (0, 3, 11),
 (0, 4, 0),
 (0, 4, 1),
 (0, 4, 2),
 (0, 4, 3),
 (0, 4, 4),
 (0, 4, 5),
 (0, 4, 6),
 (0, 4, 7),
 (0, 4, 8),
 (0, 4, 9),
 (0, 4, 10),
 (0, 4, 11),
 (0, 5, 0),
 (0, 5, 1),
 (0, 5, 2),
 (0, 5, 3),
 (0, 5, 4),
 (0, 5, 5),
 (0, 5, 6),
 (0, 5, 7),
 (0, 5, 8),
 (0, 5, 9),
 (0, 5, 10),
 (0, 5, 11),
 (0, 6, 0),
 (0, 6, 1),
 (0, 6, 2),
 (0, 6, 3),
 (0, 6, 4),
 (0, 6, 5),
 (0, 6, 6),
 (0, 6, 7),
 (0, 6, 8),
 (0, 6, 9),
 (0,

In [12]:
from itertools import groupby
def count_first_chunk_id(chunk_ids):
    return [(g, len(list(v))) for g, v in groupby(chunk_ids, lambda x: x[0])]

In [13]:
count_first_chunk_id(init_chunk_ids)

[(0, 300),
 (1, 300),
 (2, 300),
 (3, 300),
 (4, 300),
 (5, 300),
 (6, 300),
 (7, 300),
 (8, 300),
 (9, 300),
 (10, 300),
 (11, 300),
 (12, 300),
 (13, 300),
 (14, 300),
 (15, 300),
 (16, 300),
 (17, 300),
 (18, 300),
 (19, 300),
 (20, 300),
 (21, 300),
 (22, 300),
 (23, 300),
 (24, 300),
 (25, 300),
 (26, 300),
 (27, 300),
 (28, 300),
 (29, 300),
 (30, 300),
 (31, 300),
 (32, 300),
 (33, 300),
 (34, 300),
 (35, 300),
 (36, 300),
 (37, 300),
 (38, 300),
 (39, 300),
 (40, 300),
 (41, 300),
 (42, 300),
 (43, 300),
 (44, 300),
 (45, 300),
 (46, 300),
 (47, 300),
 (48, 300),
 (49, 300),
 (50, 300),
 (51, 300),
 (52, 300),
 (53, 300),
 (54, 300),
 (55, 300),
 (56, 300)]

In [14]:
ds = xr.open_zarr('http://hackathon-o.s3.jc.rl.ac.uk/' + rel_url)

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [15]:
ds.time.values[:57]

array(['2020-01-20T00:00:00.000000000', '2020-01-20T03:00:00.000000000',
       '2020-01-20T06:00:00.000000000', '2020-01-20T09:00:00.000000000',
       '2020-01-20T12:00:00.000000000', '2020-01-20T15:00:00.000000000',
       '2020-01-20T18:00:00.000000000', '2020-01-20T21:00:00.000000000',
       '2020-01-21T00:00:00.000000000', '2020-01-21T03:00:00.000000000',
       '2020-01-21T06:00:00.000000000', '2020-01-21T09:00:00.000000000',
       '2020-01-21T12:00:00.000000000', '2020-01-21T15:00:00.000000000',
       '2020-01-21T18:00:00.000000000', '2020-01-21T21:00:00.000000000',
       '2020-01-22T00:00:00.000000000', '2020-01-22T03:00:00.000000000',
       '2020-01-22T06:00:00.000000000', '2020-01-22T09:00:00.000000000',
       '2020-01-22T12:00:00.000000000', '2020-01-22T15:00:00.000000000',
       '2020-01-22T18:00:00.000000000', '2020-01-22T21:00:00.000000000',
       '2020-01-23T00:00:00.000000000', '2020-01-23T03:00:00.000000000',
       '2020-01-23T06:00:00.000000000', '2020-01-23

In [31]:
lsout = jasmin_s3.ls('sim-data/dev/Africa_km4p4_RAL3P3.n1280_GAL9_nest/data.healpix.2d.v2.z10.zarr/vas/')

In [65]:
fnames = [p.split('/')[-1] for p in lsout]
fnames

['.zarray',
 '.zattrs',
 '0.0',
 '0.1',
 '0.10',
 '0.11',
 '0.2',
 '0.3',
 '0.4',
 '0.5',
 '0.6',
 '0.7',
 '0.8',
 '0.9',
 '1.0',
 '1.1',
 '1.10',
 '1.11',
 '1.2',
 '1.3',
 '1.4',
 '1.5',
 '1.6',
 '1.7',
 '1.8',
 '1.9',
 '10.0',
 '10.1',
 '10.10',
 '10.11',
 '10.2',
 '10.3',
 '10.4',
 '10.5',
 '10.6',
 '10.7',
 '10.8',
 '10.9',
 '11.0',
 '11.1',
 '11.10',
 '11.11',
 '11.2',
 '11.3',
 '11.4',
 '11.5',
 '11.6',
 '11.7',
 '11.8',
 '11.9',
 '12.0',
 '12.1',
 '12.10',
 '12.11',
 '12.2',
 '12.3',
 '12.4',
 '12.5',
 '12.6',
 '12.7',
 '12.8',
 '12.9',
 '13.0',
 '13.1',
 '13.10',
 '13.11',
 '13.2',
 '13.3',
 '13.4',
 '13.5',
 '13.6',
 '13.7',
 '13.8',
 '13.9',
 '14.0',
 '14.1',
 '14.10',
 '14.11',
 '14.2',
 '14.3',
 '14.4',
 '14.5',
 '14.6',
 '14.7',
 '14.8',
 '14.9',
 '15.0',
 '15.1',
 '15.10',
 '15.11',
 '15.2',
 '15.3',
 '15.4',
 '15.5',
 '15.6',
 '15.7',
 '15.8',
 '15.9',
 '16.0',
 '16.1',
 '16.10',
 '16.11',
 '16.2',
 '16.3',
 '16.4',
 '16.5',
 '16.6',
 '16.7',
 '16.8',
 '16.9',
 '17.0',
 

In [58]:
chunk_ids = [p for p in fnames if not p.startswith('.z')

In [59]:
chunk_ids = set([tuple([int(v) for v in c.split('.')]) for c in chunk_ids])

In [60]:
chunk_ids

{(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (5, 10),
 (5, 11),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8),
 (8, 9),
 (8, 10),
 (8, 11),
 (9, 0),
 

In [45]:
reg = 'Africa_km4p4_RAL3P3.n1280_GAL9_nest'
url = f'https://hackathon-o.s3-ext.jc.rl.ac.uk/sim-data/dev/{reg}/data.healpix.2d.v2.z10.zarr'
ds = xr.open_zarr(url)

In [104]:
10489 / 16

655.5625

In [50]:
chunks = ds.chunks

In [53]:
list(chunks.keys())

['time', 'cell']

In [56]:
len(chunks['time']), len(chunks['cell'])

(10489, 12)

In [63]:
from itertools import product
for t, c in product(range(len(chunks['time'])), range(len(chunks['cell']))):
    print((t, c), (t, c) in chunk_ids)
    if (t, c) not in chunk_ids:
        break

(0, 0) True
(0, 1) True
(0, 2) True
(0, 3) True
(0, 4) True
(0, 5) True
(0, 6) True
(0, 7) True
(0, 8) True
(0, 9) True
(0, 10) True
(0, 11) True
(1, 0) True
(1, 1) True
(1, 2) True
(1, 3) True
(1, 4) True
(1, 5) True
(1, 6) True
(1, 7) True
(1, 8) True
(1, 9) True
(1, 10) True
(1, 11) True
(2, 0) True
(2, 1) True
(2, 2) True
(2, 3) True
(2, 4) True
(2, 5) True
(2, 6) True
(2, 7) True
(2, 8) True
(2, 9) True
(2, 10) True
(2, 11) True
(3, 0) True
(3, 1) True
(3, 2) True
(3, 3) True
(3, 4) True
(3, 5) True
(3, 6) True
(3, 7) True
(3, 8) True
(3, 9) True
(3, 10) True
(3, 11) True
(4, 0) True
(4, 1) True
(4, 2) True
(4, 3) True
(4, 4) True
(4, 5) True
(4, 6) True
(4, 7) True
(4, 8) True
(4, 9) True
(4, 10) True
(4, 11) True
(5, 0) True
(5, 1) True
(5, 2) True
(5, 3) True
(5, 4) True
(5, 5) True
(5, 6) True
(5, 7) True
(5, 8) True
(5, 9) True
(5, 10) True
(5, 11) True
(6, 0) True
(6, 1) True
(6, 2) True
(6, 3) True
(6, 4) True
(6, 5) True
(6, 6) True
(6, 7) True
(6, 8) True
(6, 9) True
(6, 

In [1]:
import zarr

In [12]:
reg = 'SAmer_km4p4_RAL3P3.n1280_GAL9_nest'
url = f'https://hackathon-o.s3-ext.jc.rl.ac.uk/sim-data/dev/{reg}/data.healpix.2d.v1.z10.zarr'


In [13]:
store = zarr.open(url)

In [14]:
tas = store['tas']

In [15]:
# Does not work for a S3 store: https://github.com/zarr-developers/zarr-python/issues/2027
tas.nchunks_initialized

0

In [16]:
tas.nchunks

41956

In [17]:
import xarray as xr

In [18]:
ds = xr.open_zarr(url)

In [20]:
ds.isel(time=1).tas.mean().compute()

<xarray.DataArray 'tas' ()> Size: 4B
array(297.82108, dtype=float32)
Coordinates:
    crs      float64 8B nan
    time     datetime64[ns] 8B 2020-01-20T01:00:00

In [21]:
zarr.__version__

'2.18.4'

In [24]:
from zarr.core.array import chunks_initialized

ModuleNotFoundError: No module named 'zarr.core.array'; 'zarr.core' is not a package

In [25]:
from zarr.util import chunks_initialized

ImportError: cannot import name 'chunks_initialized' from 'zarr.util' (/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/zarr/util.py)

In [26]:
zarr

<module 'zarr' from '/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/zarr/__init__.py'>